In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import bqplot
import IPython
import numpy
import ipywidgets


def convert_results(data, data_type = ''):
    experiment_results = []
    for experiment in data:
        if isinstance(experiment, dict):
            partial_data = list(experiment[data_type])
        else:
            partial_data = list(experiment)
        mean = numpy.mean(partial_data)
        conf = 1.96 * numpy.std(partial_data) / numpy.sqrt(len(partial_data))
        experiment_result = [mean, mean + conf, mean - conf, mean]
        experiment_results.append(experiment_result)
    return experiment_results

def plot_initialization(clustering, measures, number_of_agents):
    all_colors = ['green', 'blue', 'purple', 'orange']

    colors = {measure: all_colors[i] for i, measure in enumerate(measures)}

    for parameter, values in clustering.items():
        results = {measure: [] for measure in measures}
        figs = []

        sc_y = bqplot.LinearScale()
        sc_y_right = bqplot.LinearScale()
        sc_x = bqplot.LinearScale()

        ax_x = bqplot.Axis(label = parameter, scale = sc_x, grid_lines = 'none')
        ax_y = bqplot.Axis(label = 'Mean amount per agent', scale = sc_y, orientation = 'vertical', tick_format = '.2f')
        ax_y_right = bqplot.Axis(label = 'Walking preparedness (meters)', scale = sc_y_right, orientation = 'vertical', grid_lines = 'none', tick_format = '.0f', side = 'right')

        with ax_y_right.hold_sync():
            ax_y_right.color = colors['walking_preparedness']
            ax_y_right.label_color = colors['walking_preparedness']
        with ax_y.hold_sync():
            ax_y.color = colors['number_of_centers']
            ax_y.label_color = colors['number_of_centers']

        for measure in measures:
            for value in values:
                experiment_dir = "../DDCM-experiments/experiment_initialization_measure_%s_varying_%s/" % (measure, parameter)
                experiment_file = "%.2f_%s_%d_repeats.pkl" % (value, parameter, number_of_agents)
                with open(experiment_dir + experiment_file, "rb") as file:
                    partial_results = pickle.load(file)
                partial_results = [element if not numpy.isnan(element) else 200 for element in partial_results]

                results[measure].append(partial_results) 

            if measure in ['number_of_centers', 'number_of_charging_stations']:
                scales = {'x': sc_x, 'y': sc_y}
            else:
                scales = {'x': sc_x, 'y': sc_y_right}

            figs.append(bqplot.OHLC(x= values, y=convert_results(results[measure]), 
                marker='candle', colors = [colors[measure], colors[measure]], stroke = colors[measure], 
                stroke_width = 2.0, scales=scales, labels = [measure], display_legend = True))

        figure = bqplot.Figure(axes=[ax_x, ax_y_right, ax_y], marks=figs, 
            title = "Experiment %s varying %s" % (measure, parameter))
        toolbar = bqplot.Toolbar(figure=figure)
        complete_figure = ipywidgets.VBox([figure, toolbar, ipywidgets.HTML(value = "%s_varying_%s" % (measure, parameter))])
        IPython.display.display(complete_figure)
        
        
        
        
def plot_simulation(parameter, values, maesures, simulation_repeats, number_of_agents = -1):
    results = {measure: [] for measure in measures}
    color_training = "blue"
    color_test = "orange"

    for measure in measures:
        for value in values:
            if value not in [900, 1400, 1300, 1600]:       
                experiment_dir = "../DDCM-experiments/experiment_simulation_measures_%s_varying_%s/" % (measure, parameter)
                if parameter == 'number_of_agents':
                    experiment_file = "%s_%s_%s_simulation_repeats.pkl" % (value, parameter, simulation_repeats)
                else:
                    if parameter == 'simulation_repeats':
                        simulation_repeats = value
                    experiment_file = "%s_%s_%s_agents_%s_simulation_repeats.pkl" % (value, parameter, number_of_agents, simulation_repeats)
                    
                with open(experiment_dir + experiment_file, "rb") as file:
                    partial_results = pickle.load(file)
                results[measure].append(partial_results) 
            else:
                results[measure].append([0 for i in results[measure][-1]])

        sc_x = bqplot.LinearScale()
        sc_y = bqplot.LinearScale()
        sc_y.min = 0

        ax_x = bqplot.Axis(label=parameter, scale=sc_x)
        ax_y = bqplot.Axis(label=measure, scale=sc_y, orientation='vertical', tick_format='.2f')

        if measure in ['charging_station_validation', 'agent_validation']:
            training = bqplot.OHLC(x=values, y=convert_results(results[measure], 'training'),
                         marker='candle', scales={'x': sc_x, 'y': sc_y}, labels =['training'], display_legend = True, 
                         stroke = color_training, colors = [color_training, color_training], stroke_width = 2.0)
            test = bqplot.OHLC(x=values, y=convert_results(results[measure], 'test'),
                         marker='candle', scales={'x': sc_x, 'y': sc_y}, labels = ['test'], display_legend = True,
                         stroke = color_test, colors = [color_test, color_test], stroke_width = 2.0)
            figs = [training, test]
        else:
            figs = [bqplot.OHLC(x= values, y=convert_results(results[measure]),
                         marker='candle', scales={'x': sc_x, 'y': sc_y})]

        figure = bqplot.Figure(axes=[ax_x, ax_y], marks=figs, 
            title = "Experiment %s varying %s" % (measure, parameter))
        toolbar = bqplot.Toolbar(figure=figure)
        complete_figure = ipywidgets.VBox([figure, toolbar, ipywidgets.HTML(value = "%s_varying_%s" % (measure, parameter))])
        IPython.display.display(complete_figure)

def plot_initialization_and_simulation(parameter, values, measures, number_of_agents, simulation_repeats):
    results = {measure: [] for measure in measures}
    color_training = "blue"
    color_test = "orange"

    for measure in measures:
        for value in values:
            experiment_dir = "../DDCM-experiments/experiment_initialization_and_simulation_measures_%s_varying_%s/" % (measure, parameter)
            experiment_file = "%s_%s_%d_agents_%s_simulation_repeats.pkl" % (value, parameter, number_of_agents, simulation_repeats)
            with open(experiment_dir + experiment_file, "rb") as file:
                partial_results = pickle.load(file)
            results[measure].append(partial_results) 

        if parameter in ['distance_metric', 'weighted_centers']:
            sc_x = bqplot.OrdinalScale()
        else:
            sc_x = bqplot.LinearScale()
        sc_y = bqplot.LinearScale()
        sc_y.min = 0

        ax_x = bqplot.Axis(label=parameter, scale=sc_x)
        ax_y = bqplot.Axis(label=measure, scale=sc_y, orientation='vertical', tick_format='.2f')

        if measure in ['charging_station_validation', 'agent_validation']:
            training = bqplot.OHLC(x=values, y=convert_results(results[measure], 'training'),
                         marker='candle', scales={'x': sc_x, 'y': sc_y}, labels =['training'], display_legend = True, 
                         stroke = color_training, colors = [color_training, color_training], stroke_width = 2.0)
            test = bqplot.OHLC(x=values, y=convert_results(results[measure], 'test'),
                         marker='candle', scales={'x': sc_x, 'y': sc_y}, labels = ['test'], display_legend = True,
                         stroke = color_test, colors = [color_test, color_test], stroke_width = 2.0)
            figs = [training, test]
        else:
            figs = [bqplot.OHLC(x= values, y=convert_results(results[measure]),
                         marker='candle', scales={'x': sc_x, 'y': sc_y})]
            
        figure = bqplot.Figure(axes=[ax_x, ax_y], marks=figs, 
            title = "Experiment %s varying %s" % (measure, parameter))
        toolbar = bqplot.Toolbar(figure=figure)
        complete_figure = ipywidgets.VBox([figure, toolbar, ipywidgets.HTML(value = "%s_varying_%s" % (measure, parameter))])
        IPython.display.display(complete_figure)

def plot_distribution(parameter, values, measures, simulation_repeats):
    for measure in measures:
        for value in values:
            experiment_dir = "../DDCM-experiments/experiment_simulation_measures_%s_varying_%s/" % (measure, parameter)
            experiment_file = "%s_%s_%d_simulation_repeats.pkl" % (value, parameter, simulation_repeats)
            with open(experiment_dir + experiment_file, "rb") as file:
                results = pickle.load(file)

            scale_x = bqplot.LinearScale()
            scale_y = bqplot.LinearScale()
            ax_x = bqplot.Axis(label = measure, scale=scale_x, tick_format='0.2f')
            ax_y = bqplot.Axis(label = 'counts', scale = scale_y,
                orientation = 'vertical', grid_lines = 'solid')

            hist = bqplot.Hist(sample = results,
                scales = {'sample': scale_x, 'count': scale_y}, colors = ['green'])
            hist.bins = 50

            fig = bqplot.Figure(axes = [ax_x, ax_y], marks = [hist])
            toolbar = bqplot.Toolbar(figure = fig)
            IPython.display.display(ipywidgets.VBox([fig, toolbar]))
            
            
def plot_correlation(parameter, values, measures, simulation_repeats):
    correlation_samples = {'training': {}, 'test': {}}
    for measure in measures:
        for value in values:
            experiment_dir = "../DDCM-experiments/experiment_simulation_measures_%s_varying_%s/" % (measure, parameter)
            experiment_file = "%s_%s_%s_simulation_repeats.pkl" % (value, parameter, simulation_repeats)
            with open(experiment_dir + experiment_file, "rb") as file:
                partial_results = pickle.load(file)
            if measure == 'number_of_training_sessions_with_IDs':
                for ID, result in partial_results:
                    correlation_samples['training'][ID] = [result]
                    correlation_samples['test'][ID] = [result]
            elif measure == 'agent_validation_with_IDs':
                for validation_type in partial_results:
                    for ID, result in partial_results[validation_type]:
                        correlation_samples[validation_type][ID].append(result)

    for validation_type in ['training', 'test']:
        samples = list(correlation_samples[validation_type].values())
        samples_x = [x for x, y in samples]
        samples_y = [y for x, y in samples]

        sc_x = bqplot.LinearScale()
        sc_y = bqplot.LinearScale()

        scatt = bqplot.Scatter(x = samples_x, y = samples_y, scales = {'x': sc_x, 'y': sc_y}, colors = ['dodgerblue'])
        ax_x = bqplot.Axis(scale = sc_x, label = 'number_of_training_sessions')
        ax_y = bqplot.Axis(scale = sc_y, orientation = 'vertical', tick_format = '0.2f', label = 'agent_validation')

        fig = bqplot.Figure(marks=[scatt], axes=[ax_x, ax_y], title = 
            'Correlation of number of training sessions and agent validation (%s error)' % validation_type)
        toolbar = bqplot.Toolbar(figure = fig)
        IPython.display.display(ipywidgets.VBox([fig, toolbar]))


## 1.1 Experiment initialization: Effects of clustering parameters on clustering metrics

In [ ]:
clustering = {}

min_value, step_value, max_value = 0.1, 0.2, 3
values = numpy.arange(min_value, max_value + step_value, step_value)
clustering['clustering_birch_threshold'] = values

min_value, step_value, max_value = 1.0, 1.0, 20.0
values = numpy.arange(min_value, max_value + step_value, step_value)
clustering['clustering_lon_lat_scale'] = values

min_value, step_value, max_value = 0.01, 0.02, 0.24
values = numpy.arange(min_value, max_value + step_value, step_value)
clustering['threshold_fraction_sessions'] = values

min_value, step_value, max_value = 5, 5, 50
values = range(min_value, max_value + step_value, step_value)
clustering['minimum_nr_sessions_center'] = values

min_value, step_value, max_value = 5, 5, 50
values = range(min_value, max_value + step_value, step_value)
clustering['minimum_nr_sessions_cs'] = values

measures = ['number_of_centers', 'number_of_charging_stations', 'walking_preparedness']
number_of_agents = 2000

plot_initialization(clustering, measures, number_of_agents)

## 1.2 Experiment initialization and simulation: Effect of max gap sessions agent on validation metrics and time metrics

In [ ]:
parameter = 'max_gap_sessions_agent'
min_, step_, max_ = 10, 10, 100
values = range(min_, max_ + step_, step_)
measures = ['agent_validation', 'charging_station_validation', 'time_per_simulation', 'time_per_initialization']
number_of_agents, simulation_repeats = 100, 100

plot_initialization_and_simulation(parameter, values, measures, number_of_agents, simulation_repeats)

## 1.3 Experiment initialization and simulation: Effect of bin size dist on validation metrics and time metrics


In [ ]:
parameter = 'bin_size_dist'
min_bin_size, bin_size_step, max_bin_size = 10, 10, 360
bin_size_range = range(min_bin_size, max_bin_size + bin_size_step, bin_size_step)
values = [bin_size for bin_size in bin_size_range if 1440 % bin_size == 0]
measures = ['agent_validation', 'charging_station_validation', 'time_per_simulation', 'time_per_initialization']
number_of_agents, simulation_repeats = 100, 100

plot_initialization_and_simulation(parameter, values, measures, number_of_agents, simulation_repeats)

## 1.4 Experiment simulation: Effect of habit probability on validation metrics and time metrics

In [ ]:
parameter = 'habit_probability'
min_, step_, max_ = 0, 0.2, 1
values = numpy.arange(min_, max_ + step_, step_)
measures = ['charging_station_validation', 'agent_validation', 'time_per_simulation']
number_of_agents, simulation_repeats = 100, 100

plot_simulation(parameter, values, measures, simulation_repeats, number_of_agents = number_of_agents)

## 1.5 Experiment simulation: Effect of time retry center on validation metrics and time metrics

In [ ]:
parameter = 'time_retry_center'
min_, step_, max_ = 30, 30, 180
values = range(min_, max_ + step_, step_)
measures = ['charging_station_validation', 'agent_validation', 'time_per_simulation']
number_of_agents, simulation_repeats = 100, 100

plot_simulation(parameter, values, measures, simulation_repeats, number_of_agents = number_of_agents)

## 1.6 Experiment initialization and simulation: Effect of default walking preparedness metrics and time metrics

In [ ]:
parameter = 'default_walking_preparedness'
min_, step_, max_ = 50, 50, 300
values = range(min_, max_ + step_, step_)
measures = ['agent_validation', 'charging_station_validation', 'time_per_simulation', 'time_per_initialization']
number_of_agents, simulation_repeats = 100, 100

plot_initialization_and_simulation(parameter, values, measures, number_of_agents, simulation_repeats)

## 1.7 Experiments simulation: Effect of warmup period in days on validation metrics and time metrics

In [ ]:
parameter = 'warmup_period_in_days'
min_, step_, max_ = 5, 30, 155
values = range(min_, max_ + step_, step_)
measures = ['charging_station_validation', 'agent_validation', 'time_per_simulation']
number_of_agents, simulation_repeats = 100, 100

plot_simulation(parameter, values, measures, simulation_repeats, number_of_agents = number_of_agents)

## 1.8 Experiments initialization and simulation: Effect  of  distance  measure on  validation metrics and time metrics

In [ ]:
parameter = 'distance_metric'
values = ['as_the_crow_flies', 'walking']
measures = ['agent_validation', 'charging_station_validation', 'time_per_simulation', 'time_per_initialization']
number_of_agents, simulation_repeats = 100, 100

plot_initialization_and_simulation(parameter, values, measures, number_of_agents, simulation_repeats)

## 1.9 Experiments initialization and simulation: Effect of weighted vs unweighted centers on validation metrics and time metrics and distance and walking preparedness

In [ ]:
parameter = 'weighted_centers'
values = [True, False]
measures = ['agent_validation', 'charging_station_validation',
    'time_per_simulation', 'time_per_initialization', 'maximum_distance',
    'walking_preparedness']
number_of_agents, simulation_repeats = 100, 100

plot_initialization_and_simulation(parameter, values, measures, number_of_agents, simulation_repeats)

## 1.10 Distribution of walking preparedness

In [ ]:
parameter = 'number_of_agents'
values = [2244]
measures = ['walking_preparedness'] # 'maximum_distance' only after new agents created
simulation_repeats = 0

plot_distribution(parameter, values, measures, simulation_repeats)

## 1.11 Effect of simulation repeats on validation metrics and time metrics

In [ ]:
parameter = 'simulation_repeats'
min_, step_, max_ = 10, 20, 310
values = range(min_, max_ + step_, step_)
measures = ['charging_station_validation', 'agent_validation', 'time_per_simulation']
number_of_agents = 100
simulation_repeats = -1

plot_simulation(parameter, values, measures, simulation_repeats, number_of_agents = number_of_agents)

## 2.1 Experiments simulation: Effect of number of agents on validation metrics and time metrics

In [ ]:
parameter = 'number_of_agents'
min_nr_agents, agent_step, max_nr_agents = 100, 100, 2200
values = range(min_nr_agents, max_nr_agents + agent_step, agent_step)
measures = ['charging_station_validation', 'agent_validation', 'time_per_simulation']
simulation_repeats = 30

plot_simulation(parameter, values, measures, simulation_repeats)

## 2.2 Correlation of number of training sessions and agent validation

In [ ]:
parameter = 'number_of_agents'
values = [2000]
measures = ['number_of_training_sessions_with_IDs', 'agent_validation_with_IDs']
simulation_repeats = 30

plot_correlation(parameter, values, measures, simulation_repeats)